# VEGAS implementation by Timur Khuzin

Sources:
1. https://e-maxx.ru/bookz/files/numerical_recipes.pdf, pp. 414-416
2. http://numerical.recipes/webnotes/nr3web9.pdf

## Implementation

In [1]:
import numpy as np
import numpy.random
from sklearn.preprocessing import normalize

class Vegas:
    def __init__(self, region, f, rand=np.random.RandomState(seed=164), steps_per_dim=1000):
        '''
        Constructor.
            region          is matrix 2xD, where D is dimension number. region[0,:] is lower bounds, region[1,:] is upper
            f               is integrated function. Must accept numpy vector with len D 
            rand            random float generator in range[0;1)
            steps_per_dim   maximal number of increments per axis
        '''
        self.f = f
        self.rand = rand
        self.steps_per_dim = steps_per_dim
        self.shift = region[0,:]
        self.region = np.copy(region)
        self.region[0, :] -= self.shift
        self.region[1, :] -= self.shift
        self.dimensions = len(region[0,:])
        
    def warm_up(epochs, call_per_epoch):
        '''
        Prepare field for integration.
        params:
            epochs          number of iterations of algorithm (calculations of integral)
            call_per_epoch  how many checks needed to calc integral in each iteration
        '''
        steps_per_dim = self.steps_per_dim
        
        # initiate field with equal subintervals
        field = np.zeros((self.dimensions, steps_per_dim))
        for x in range(self.dimensions):
            lattice = np.arange(0,self.region[1,x],self.region[1,x]/steps_per_dim)
            field[x, :] = lattice
        
        # weights
        probs = np.ones((self.dimensions, steps_per_dim))
        probs = normalize(probs, axis=1)
        
        total_range = self.region[1,:]
        for e in range(epochs):
            # new weights which will be used in next one
            weight_update = np.clone(probs)
            probs_reverted = 1-probs
            for c in range(call_per_epoch):
                chosen_indexes = self.rand.randint(0, self.steps_per_dim, self.dimensions)
                ranges = self.rand.rand(self.dimensions)
                argument = np.choose(chosen_indexes,field.T) + np.choose(chosen_indexes, probs_reverted.T)*ranges
                argument += self.shift
                weight_change = np.abs(self.f(argument))
                for i,j in zip(range(self.dimensions, chosen_indexes)):
                    weight_update[i,j] += weight_change
            probs = normalize(weight_update, axis=1)
            probs_reverted = 1 - probs
            behind_by_lattice = np.zeros(self.dimensions)
            for step in range(steps_per_dim):
                field[:, step] = behind_by_lattice
                behind_by_lattice += probs_reverted[:, step] * total_range 
                
        
    def vegas_integral(epochs, call_per_epoch):
        '''
        Implements vegas integral.
        params:
            epochs          number of iterations of algorithm (calculations of integral)
            call_per_epoch  how many checks needed to calc integral in each iteration

        returns object with fields: 
            integral        float value of calculated integral
            std_deviation   standard deviation of calculated integral
            chi2a           look Chi-squared test
        '''

        probs_reverted = np.transpose(1-probs)
        integrals = np.zeros(epochs)
        for iteration in range(epochs):
            integr = 0.0
            for c in range(call_per_epoch):
                chosen_indexes = self.rand.randint(0, self.steps_per_dim, self.dimensions)
                ranges = self.rand.rand(self.dimensions)
                chose_volume = np.choose(chosen_indexes, probs_reverted.T)
                argument = np.choose(chosen_indexes,field.T) + chose_volume*ranges
                argument += self.shift
                value = self.f(argument)
                integr += value
            integr /= call_per_epoch
            integrals[iteration] = integr
        

        # return result
        class answer: pass
        ans = answer()
        ans.integral = integral
        ans.std_deviation = std_deviation
        ans.chi2a = chi2a
        return ans

In [2]:
a = np.zeros((2,5))
print(a)
a[1,:] = 1
print(a)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1.]]


In [4]:
normalize(np.array([[1,2,3],[4,5,6],[7,8,9]]), axis=1),normalize(np.array([[1,2,3],[4,5,6],[7,8,9]]), axis=0)

(array([[0.26726124, 0.53452248, 0.80178373],
        [0.45584231, 0.56980288, 0.68376346],
        [0.50257071, 0.57436653, 0.64616234]]),
 array([[0.12309149, 0.20739034, 0.26726124],
        [0.49236596, 0.51847585, 0.53452248],
        [0.86164044, 0.82956136, 0.80178373]]))

In [27]:
np.array([[1,2,3],[4,5,6],[7,8,9]]),\
np.choose(np.array([0,2,2]), np.array([[1,2,3],[4,5,6],[7,8,9]]).T)

(array([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]), array([1, 6, 9]))

In [22]:
probs = np.ones((4,5))
normalize(probs, axis=1)

array([[0.4472136, 0.4472136, 0.4472136, 0.4472136, 0.4472136],
       [0.4472136, 0.4472136, 0.4472136, 0.4472136, 0.4472136],
       [0.4472136, 0.4472136, 0.4472136, 0.4472136, 0.4472136],
       [0.4472136, 0.4472136, 0.4472136, 0.4472136, 0.4472136]])